<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-27 09:42:38
-------------------
qualified stocks: 91
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  9.00 L
Current:  1.37 C
-------------------
Today PnL: -1.57 L (-1.13%)
Current PnL: -19.93 L (-13.41%)
CY Booked + Current PnL: -10.89 L (-7.33%)
-------------------
Total profit:  2.96 L
Total loss:  -22.89 L
-------------------
Total Booked + Current PnL: 16.09 L (13.22%)
Total Booked PnL: 36.02 L (29.6%)
Curr Year Booked PnL: 9.04 L (6.58%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.16 C
Est FTT PnL: 78.32 L (57.04%)
Deployed:  1.22 C
Current:  1.37 C
CAGR/XIRR %: 7.71%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HINDALCO,651.95,756.01,-18.25,H-LC,7.35,112640.0,8328.0,8324.0,-1.66,7.98,7.39,15.96,9.0,1.00,0.88,26.23,X5K,ATH,METALS
53,MEDANTA,1087.93,1486.00,13.77,H-MC,14.69,162412.0,36212.0,9972.0,0.27,28.69,6.14,36.59,108.0,3.63,1.27,44.86,XY24,NTT,HEALTHCARE
4,APOLLOHOSP,7098.95,8285.00,-14.64,H-LC,8.10,170632.0,14455.0,11637.0,-1.21,9.26,6.82,16.71,22.0,1.24,1.33,28.31,X40N,BTT,HEALTHCARE
66,SBIN,760.30,863.00,-13.25,M-LC,4.02,214226.0,12746.0,14460.0,-0.99,6.33,6.75,13.51,60.0,0.88,1.67,19.72,XY25,NTT,BANKS
81,TTKPRESTIG,769.29,770.00,94.66,M-SC,8.63,83840.0,-16937.0,17028.0,-2.46,-16.81,20.31,0.09,245.0,-0.99,0.65,10.15,OX40N,NTT,DURABLES


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,KPIGREEN,497.21,731.35,5.57,H-SC,11.08,125748.0,451.0,58548.0,-2.16,0.36,46.56,47.09,141.0,0.01,0.98,57.38,MH,ATH,POWER
59,RAJOOENG,102.50,143.33,-31.64,H-SC,10.00,103500.0,1000.0,39827.0,-0.43,0.98,38.48,39.83,114.0,0.03,0.81,2.48,AR,ATH,MISC


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
45,ITC,411.90,452.0,-38.77,H-LC,3.05,146819.0,-3113.0,17706.0,1.00,-2.08,12.06,9.74,4.0,-0.18,1.14,4.03,X40,NTT,FMCG
51,LTIM,5564.16,7230.2,-6.36,H-LC,3.25,185508.0,-14802.0,74778.0,-2.65,-7.39,40.31,29.94,16.0,-0.20,1.45,28.16,X200,ATH,IT
55,NESTLEIND,1134.30,1377.0,-11.91,H-LC,3.32,271253.0,5827.0,50968.0,0.54,2.20,18.79,21.40,11.0,0.11,2.11,9.44,XY25,NTT,FMCG
20,CIPLA,1492.70,1795.0,-19.09,H-LC,4.20,215994.0,11494.0,29915.0,-1.54,5.62,13.85,20.25,10.0,0.38,1.68,14.84,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-13.06,H-LC,6.44,169524.0,-31862.0,119481.0,0.00,-15.82,70.48,43.51,27.0,-0.27,1.32,18.01,X40,BTT,PAINTS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
15,BRITANNIA,5502.66,6362.67,14.35,H-LC,8.69,207036.0,8940.0,22029.0,3.61,4.51,10.64,15.63,26.0,0.41,1.61,27.35,XY25,ATH,FMCG
45,ITC,411.90,452.00,-38.77,H-LC,3.05,146819.0,-3113.0,17706.0,1.00,-2.08,12.06,9.74,4.0,-0.18,1.14,4.03,X40,NTT,FMCG
67,SFL,1016.35,1287.00,40.09,M-SC,10.39,184986.0,-77232.0,147064.0,0.94,-29.45,79.50,26.63,229.0,-0.53,1.44,15.01,XY24,NTT,MISC
79,TITAGARH,1097.23,1548.00,-7.36,H-SC,9.63,151812.0,-40203.0,119081.0,0.75,-20.94,78.44,41.08,158.0,-0.34,1.18,28.49,XY24,NTT,ENGINEERING
70,SIS,477.00,528.00,2225.39,H-SC,5.60,62560.0,-18530.0,27201.0,0.74,-22.85,43.48,10.69,156.0,-0.68,0.49,26.79,OX40N,NTT,MISC


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,EASEMYTRIP,18.12,26.40,-15.15,M-SC,28.50,81674.0,-95141.0,175934.0,-4.99,-53.81,215.41,45.70,197.0,-0.54,0.64,0.00,XY24,NTT,TRAVEL
85,VAIBHAVGBL,350.21,521.00,34.44,H-SC,11.16,89647.0,-62344.0,136470.0,-4.22,-41.02,152.23,48.77,125.0,-0.46,0.70,4.34,XR,NTT,JEWELLERY
69,SIEMENS,4406.90,7969.85,-5.18,H-LC,33.21,76250.0,-33922.0,122999.0,-3.62,-30.79,161.31,80.85,15.0,-0.28,0.59,12.30,AR,ATH,ELECTRICAL
86,VALIANTORG,512.64,838.00,-306.49,H-SC,13.11,87750.0,-45536.0,130133.0,-3.52,-34.16,148.30,63.47,139.0,-0.35,0.68,47.09,XR,NTT,CHEMICALS
10,BANDHANBNK,214.25,400.00,-11.26,H-SC,8.48,191957.0,-56573.0,272041.0,-3.48,-22.76,141.72,86.70,151.0,-0.21,1.50,28.15,XY24,NTT,BANKS


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TTKPRESTIG,769.29,770.0,94.66,M-SC,8.63,83840.0,-16937.0,17028.0,-2.46,-16.81,20.31,0.09,245.0,-0.99,0.65,10.15,OX40N,NTT,DURABLES
82,UJJIVANSFB,52.77,60.0,80.86,H-SC,15.43,114480.0,-27999.0,47521.0,-3.09,-19.65,41.51,13.70,163.0,-0.59,0.89,35.12,OX40N,NTT,BANKS
19,CERA,7989.07,9475.0,-20.53,H-SC,3.07,113229.0,-30574.0,57317.0,-2.73,-21.26,50.62,18.60,149.0,-0.53,0.88,24.95,OX40N,NTT,CERAMICS
48,KANSAINER,299.63,340.0,-69.25,H-SC,3.64,211491.0,-58176.0,94515.0,0.19,-21.57,44.69,13.47,138.0,-0.62,1.65,6.97,XY24,NTT,PAINTS
41,INDIGOPNTS,1407.73,1408.0,108.15,M-SC,1.73,136648.0,-37911.0,37947.0,-2.17,-21.72,27.77,0.02,221.0,-1.00,1.06,18.86,OX40N,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TCS,3794.03,4998.00,-25.47,H-LC,15.93,246246.0,-49688.0,143586.0,0.52,-16.79,58.31,31.73,1.0,-0.35,1.92,5.13,X40,BTT,IT
43,INFY,1461.46,2275.00,-16.73,H-LC,11.90,273118.0,11517.0,134101.0,-0.41,4.40,49.10,55.67,3.0,0.09,2.13,10.78,X40,BTT,IT
45,ITC,411.90,452.00,-38.77,H-LC,3.05,146819.0,-3113.0,17706.0,1.00,-2.08,12.06,9.74,4.0,-0.18,1.14,4.03,X40,NTT,FMCG
87,VBL,492.64,671.64,-10.10,H-LC,8.65,273035.0,7995.0,88300.0,-1.65,3.02,32.34,36.33,5.0,0.09,2.13,16.72,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-42.30,H-LC,12.85,195858.0,-20661.0,113558.0,-1.54,-9.54,57.98,42.91,7.0,-0.18,1.53,2.57,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1087.93,1486.0,13.77,H-MC,14.69,162412.0,36212.0,9972.0,0.27,28.69,6.14,36.59,108.0,3.63,1.27,44.86,XY24,NTT,HEALTHCARE
27,DMART,3764.92,5201.0,-7.27,H-LC,11.16,231525.0,47044.0,23315.0,-0.01,25.50,10.07,38.14,38.0,2.02,1.80,38.81,X40N,NTT,FMCG


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,MEDANTA,1087.93,1486.00,13.77,H-MC,14.69,162412.0,36212.0,9972.0,0.27,28.69,6.14,36.59,108.0,3.63,1.27,44.86,XY24,NTT,HEALTHCARE
14,BLUESTARCO,1646.70,2326.38,8.41,H-MC,4.81,187200.0,22530.0,45433.0,-1.70,13.68,24.27,41.28,89.0,0.50,1.46,22.76,X40N,ATH,AC
49,KPIGREEN,497.21,731.35,5.57,H-SC,11.08,125748.0,451.0,58548.0,-2.16,0.36,46.56,47.09,141.0,0.01,0.98,57.38,MH,ATH,POWER
88,VOLTAS,1278.28,1918.49,-3.74,H-MC,6.76,201900.0,10158.0,85868.0,-0.84,5.30,42.53,50.08,99.0,0.12,1.57,12.46,XY25,ATH,AC
34,HEROMOTOCO,4311.81,5949.07,-4.42,H-MC,14.03,177622.0,26709.0,30604.0,0.22,17.70,17.23,37.97,93.0,0.87,1.38,47.48,AR,ATH,AUTO


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,4311.81,5949.07,-4.42,H-MC,14.03,177622.0,26709.0,30604.0,0.22,17.70,17.23,37.97,93.0,0.87,1.38,47.48,AR,ATH,AUTO
40,INDIAMART,2327.09,4850.92,-49.60,H-SC,12.38,140450.0,17114.0,116644.0,0.47,13.88,83.05,108.45,119.0,0.15,1.09,39.33,AR,ATH,MISC
89,WHIRLPOOL,1219.98,2270.00,-42.94,M-SC,5.56,96390.0,4892.0,73864.0,-1.82,5.35,76.63,86.07,223.0,0.07,0.75,39.44,XR,NTT,DURABLES
90,WIPRO,243.46,420.00,-11.07,M-LC,7.77,156085.0,5140.0,104312.0,-0.82,3.41,66.83,72.51,53.0,0.05,1.22,9.63,XR,NTT,IT
59,RAJOOENG,102.50,143.33,-31.64,H-SC,10.00,103500.0,1000.0,39827.0,-0.43,0.98,38.48,39.83,114.0,0.03,0.81,2.48,AR,ATH,MISC


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.00,-43.66,L-SC,27.10,82406.0,-31143.0,71191.0,-0.16,-27.43,86.39,35.27,268.0,-0.44,0.64,101.69,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-3.67,H-SC,16.15,89642.0,-11321.0,105849.0,-1.78,-11.21,118.08,93.63,148.0,-0.11,0.70,33.52,SR,ATH,CHEMICALS
52,MASFIN,326.60,398.61,-19.15,H-SC,12.53,92040.0,-5940.0,27548.0,-1.84,-6.06,29.93,22.05,152.0,-0.22,0.72,33.79,XR,ATH,FINANCE
35,HINDALCO,651.95,756.01,-18.25,H-LC,7.35,112640.0,8328.0,8324.0,-1.66,7.98,7.39,15.96,9.0,1.00,0.88,26.23,X5K,ATH,METALS
82,UJJIVANSFB,52.77,60.00,80.86,H-SC,15.43,114480.0,-27999.0,47521.0,-3.09,-19.65,41.51,13.70,163.0,-0.59,0.89,35.12,OX40N,NTT,BANKS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,59.14,80.00,-43.66,L-SC,27.10,82406.0,-31143.0,71191.0,-0.16,-27.43,86.39,35.27,268.0,-0.44,0.64,101.69,XR,NTT,HOTELS
27,DMART,3764.92,5201.00,-7.27,H-LC,11.16,231525.0,47044.0,23315.0,-0.01,25.50,10.07,38.14,38.0,2.02,1.80,38.81,X40N,NTT,FMCG
34,HEROMOTOCO,4311.81,5949.07,-4.42,H-MC,14.03,177622.0,26709.0,30604.0,0.22,17.70,17.23,37.97,93.0,0.87,1.38,47.48,AR,ATH,AUTO
67,SFL,1016.35,1287.00,40.09,M-SC,10.39,184986.0,-77232.0,147064.0,0.94,-29.45,79.50,26.63,229.0,-0.53,1.44,15.01,XY24,NTT,MISC
14,BLUESTARCO,1646.70,2326.38,8.41,H-MC,4.81,187200.0,22530.0,45433.0,-1.70,13.68,24.27,41.28,89.0,0.50,1.46,22.76,X40N,ATH,AC


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.96
1,25,42.09
2,50,72.29


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    38.30
LC    34.60
MC    27.13
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.26
X40N     14.37
X40      14.20
XY25      9.97
XR        9.24
AR        8.72
OX40N     7.59
X5K       2.34
MH        1.75
X200      1.45
SR        1.14
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.10
H-MC    24.17
H-SC    23.16
M-SC    13.55
M-LC     7.43
M-MC     2.62
L-SC     1.59
L-LC     1.07
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.93,-1.28,34.80
IT,12.46,-16.89,75.27
BANKS,7.67,-16.02,65.28
FINANCE,7.52,-25.10,71.23
MISC,6.11,-17.18,84.34
PAINTS,5.69,-19.46,43.00
ELECTRICAL,4.88,-12.29,60.93
AUTO,4.72,-11.65,60.23
AC,3.79,0.41,33.32


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2884374.0,23
XR,1108009.0,13
AR,1089764.0,10
X40,780646.0,10
X40N,521451.0,11
OX40N,485618.0,10
XY25,459954.0,7
SR,197004.0,2
X5K,139233.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2803380.0,27
M-SC,1506103.0,17
H-MC,1280490.0,18
H-LC,1110806.0,17
M-LC,426903.0,5
M-MC,341353.0,2
L-SC,257323.0,3
L-MC,59375.0,1
L-LC,46555.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      946101.0      6
M-SC       XY24      878795.0      7
H-SC       AR        663409.0      5
           XR        534098.0      6
H-MC       XY24      491018.0      5
H-LC       X40       470345.0      5
M-MC       XY24      341353.0      2
H-LC       AR        236557.0      2
H-MC       X40       224893.0      4
H-SC       OX40N     209390.0      4
H-MC       XY25      201319.0      2
H-LC       X40N      197920.0      5
H-SC       SR        197004.0      2
           X40N      194830.0      3
M-LC       XY24      177222.0      2
L-SC       XR        166956.0      2
M-SC       AR        159194.0      2
           OX40N     134104.0      4
M-LC       X5K       130909.0      1
H-MC       X40N      128701.0      3
M-SC       XY25      124623.0      1
           XR        123979.0      2
H-MC       XR        119289.0      1
M-LC       XR        104312.0      1
L-SC       OX40N      90367.0      1
M-SC       X40        85408.0      1
H-LC       X200       74778.0      1
           XY25       72997.0      2
L-MC       XR         59375.0      1
H-SC       MH         58548.0      1
H-MC       OX40N      51757.0      1
H-LC       XY24       49885.0      1
L-LC       XY25       46555.0      1
H-MC       MH         32909.0      1
           AR         30604.0      1
M-LC       XY25       14460.0      1
H-LC       X5K         8324.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 36.0 seconds
